# Religion and Evolution in Early NZ Newspapers

This notebook sets out an example of the kind of digital humanities use to which the dataset developed in my DATA601 project can be put.

In [1]:
import copy

from jupyter_dash import JupyterDash
import dash_cytoscape as cyto
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

from gensim import corpora
from gensim.models import LdaMulticore, TfidfModel
from gensim.matutils import corpus2csc

import pandas as pd

from nltk.text import Text, TextCollection
from nltk.tokenize import RegexpTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords, words

import NL_helpers
import NL_topicmodels

TOKENIZER = RegexpTokenizer(r"[A-Za-z']+")
STOPWORDS = set(stopwords.words('english'))

In [12]:
# Run cell to reload NL_helpers and NL_topicmodels if they have been changed.
from importlib import reload
reload(NL_helpers)
reload(NL_topicmodels)

<module 'NL_topicmodels' from '/home/joshua/hdd/Documents/MADS/DATA601/NL_topicmodels.py'>

## Background

One of the main intellectual developments which took place within the time period covered by the Papers Past newspaper open data pilot is the development and publication of the theory of evolution by natural selection (1859 onwards). This engendered a lot of public debate, including debate over the consequences of these ideas for religious beliefs and believers. Ballantyne (2012) and Crane (2013) provide clear evidence that this discussion was carried out in New Zealand newspapers. Both the important of the issue and the knowledge that it is present in the dataset make it a good test subject for the corpus.

One question to ask about this data is what attitudes to evolution are present in the dataset. A sophisticated approach would be to run some kind of algorithmic sentement analysis. However, simpler methods can be appropriate. In this case, we will use concordancing.

A hypothesis to be working with can be generated by appeal to historical work on the interaction between evolutionary thought and Christianity. 

* 'the book of nature': study of creation is a way to learn about the creator. (351) (word and work - Dawson 356-357)
* 'warfare'/'conflict' (352) argues that White's book was written IN ORDER to defend idea of secular university - so connected with other topic.
* Within the sciences: Restricted Darwinism, Full-blown Darwinism, rejection of Darwinism (353-354).
* Science-religion: conversion: giving up religion and swearing allegiance to science, (the metaphor of warfare dominates.)
* Sinking of natural theology.
* Language of conversion


Marks of continuity (355)?
Crisis over 'higher criticism' happens at almost the same time. (355)
Spencer as creating a kind of religion.

## Loading the and Processing the Dataset

The following dataset was produced by using a Naive Bayes classifier trained on the labelled dataset to distinguish between religion-science and all other tags in the philosophy dataset. 

In [2]:
rel_df = pd.read_pickle('pickles/rel_v2_philoso_df.tar.gz')

In [5]:
rel_df

,Title,Text,Newspaper,Date,Text as String
LT_18940109_ARTICLE7,THE TASK OF THE BIOLOGIST.,[No. 11. When chemistry had finished _ shaping...,LT,18940109,No. 11. When chemistry had finished _ shaping;...
LT_18940115_ARTICLE5,THE THEORY OF EVOLUTION.,[SIGNIFICANT RUDIMENTS. [BY W.G.P.] No. lII.' ...,LT,18940115,SIGNIFICANT RUDIMENTS. [BY W.G.P.] No. lII.' '...
LT_18940129_ARTICLE48,THE THEORY OF EVOLUTION.,[A CHANGED WORLD. [BY W.G.P.] No. V. The bitte...,LT,18940129,A CHANGED WORLD. [BY W.G.P.] No. V. The bitter...
LT_18940219_ARTICLE18,HERESY.,"[TO THE EDITOR. Sir,— The action of the Presby...",LT,18940219,"TO THE EDITOR. Sir,— The action of the Presbyt..."
LT_18940219_ARTICLE24,REV JAMES GIBB ON CHRISTIAN DUTY.,"[At St Andrew’s Presbyterian Church, yesterday...",LT,18940219,"At St Andrew’s Presbyterian Church, yesterday ..."
...,...,...,...,...,...
ODT_18771031_ARTICLE30,THE INDIAN FAMINE.,[TO THE TJDITOR. Sir—l think I have_ fair caus...,ODT,18771031,TO THE TJDITOR. Sir—l think I have_ fair cause...
ODT_18771107_ARTICLE32,POPULAR CULTURE.,"[The. discourse of Mi* J^hn M >r!ev on "" I'opu...",ODT,18771107,"The. discourse of Mi* J^hn M >r!ev on "" I'opul..."
ODT_18790115_ARTICLE20,MR BRIGHT AND THE ELDERS. TO THE EDITOR.,"[Sib,—However objectionable Mr Bright's title ...",ODT,18790115,"Sib,—However objectionable Mr Bright's title m..."
ODT_18790118_ARTICLE23,UNTITLED,"[Sir,—Before the public can fairly under stand...",ODT,18790118,"Sir,—Before the public can fairly under stand ..."


We will need to tokenise this dataset to use nltk concordancing. To get the full context, we can't use stop words.

In [3]:
rel_df['Tokenised'] = rel_df['Text as String'].apply(lambda x: TOKENIZER.tokenize(x.lower()))

## Concordancing

Using the methods presented above, we can use concordancing to look for some markers of each of these attitudes to evolution.

In [7]:
full_corpus = TextCollection(rel_df['Tokenised'])

As the NLTK `.concodance()` function does not allow for phrases to be searched, the following helper function will be useful:

In [8]:
def concordance_phrase(text, left_context, key_term, right_context, width=100):
    results = text.concordance_list(key_term, width=width, lines=10000)
    desired = [True] * len(results)    
    left_list = left_context.split()   
    right_list = right_context.split()    
    for i, line in enumerate(results):
        # Handle left context
        for j, word in enumerate(left_list):
            position = len(line[0])-len(left_list)+j
            if desired[i] and not line[0][position] == word:
                desired[i] = False
         # Handle right contex
        for j, word in enumerate(right_list):
            if desired[i] and not line[2][j] == word:
                desired[i] = False
    final_results = [results[i] for i in range(len(results)) if desired[i]==True]
    
    # Print formatted concordance
    if len(final_results) == 0:
        print('No results found.')
    for result in final_results:
        print(result.line)            

In [9]:
concordance_phrase(full_corpus, '', 'warfare', 'of', width=110)

s will then instead of the jangle of words and the warfare of dogmas the energy of christians will be devoted
e than the details of mr darwin but this ceaseless warfare of the champions of that faith which we at least h
ws during his recent visit to america r review the warfare of science by andrew dickson whitb ll d president 
 may ba conceivably created or strengthened in tha warfare of apologetics that ia eot necessarily by reason o
 fresh to the subject the prime discovery in naval warfare of breaking the enemy's line was made not by a sea
 fresh to the subject the prime discovery in naval warfare of breaking the enemy's line was made not by a sea
re at the pres byterian church last evening on the warfare of science and religion there was a moderately goo
ve ment and progress of the human family hence the warfare of a few simple folk en gaged at present in auckla
e than the details of mr darwin but this ceaseless warfare of the champions of that faith which we at least h
 in his nb

In [10]:
concordance_phrase(full_corpus, '', 'conflict', 'of', width=110)

lex phenomena all the agencies set at work by such conflict of forces and never until this master key to the c
monstrated the indestructibility of energy and tho conflict of the opposing scientific schools ceased for ever
o distinct churches in which not only division but conflict of minds and interests has been manifested while a
i shall never know what will be the outcome of the conflict of european nationalities w hat will be the fate o
e and intol erable we read every day o the endless conflict of seltish interests of strikes and lock ontp ol t
mble but when antagonism rises against it'the very conflict of forces brings out the latent powe iu full energ
he god within him the god that shall outride every conflict of infidelity every conflict of siu and vice and l
t shall outride every conflict of infidelity every conflict of siu and vice and lust and crime aud ignorance a
l continue to make men wiser and purer dr draper's conflict of science and religion is a very interesting book
 

In [169]:
concordance_phrase(full_corpus, '', 'antagonism', 'between', width=110)

worthy of all support so far from there being any antagonism between it and borderland that magizine will have
ion that there are gratifying signs of decreasing antagonism between the religions and scientific worlds the t
perfectly true to say that there is and can be no antagonism between the church and science while at the same 
t and clash and hence arises what is tnown ns the antagonism between science aud religion but the antagonism i
ant interpretations of the bible rest the fancied antagonism between scientific progress and religious truth a
 over to he enforced by the police the so called' antagonism between religion and science will disappear when 
in papers i have not in my lectures presented any antagonism between genesis and geology unless a fair present
ned themselves to the main ques tion is there any antagonism between true christian revelation and true evolut
perfectly true to say that there is and can be no antagonism between the church and science whilo at the same 
p

In [170]:
concordance_phrase(full_corpus, 'the', 'creator', '', width=110)

 ring out the more concordantly shall we blame the creator for the present dissonant condi tions of life p ha
e declare the cosmos as a whole to be immortal the creator when he launched the infinite did not do his work 
hrist s resur rection in preference to that of the creator s rest in reply to j c i may state thatthisistnere
ion of thought the thought of the supreme mind the creator who said let there he light and there was light fr
 which ssth is an example apparently tliink of the creator as a kind of apprentice god whose power is so limi
i would ask him which is first the creature or the creator the invention or i the inventor however underlying
 clay to earth taught clay it was the voice of the creator calling to his children to learn the new lesson of
e created will to oppose itself to the laws of the creator this led up to what was meant by the evil one whom
orude as ulao would be tueir ideas of j him ba the creator of the world atill theyi were true and as their in
ays that t

In [171]:
concordance_phrase(full_corpus, '', 'theism', 'and', width=110)

ool says it is fast evaporating into a mere natural theism and is thus showing us what as a governing power na
ell you say that morals are a farce unless based on theism and a future life is this meant to be what logician
e he wrongly assumes that i think only believers in theism and a future state are moral and he uses such words
te individualities form or make one with re gard to theism and atheism atheism is really not worth the answer 
gtoniano the loan is not often acknowledged but pan theism and pantheistic interpretations of the universe how
nism all that capt hutton apparently can say io hat theism and darwinism are not neces larily antagonistic but
ng up this posi tion they preserved themselves from theism and materialism again it was believed by many of th
difficult question of the relation of evolution and theism and yet such a question must really bs the prelimin
kly say in what re spect and to what extent his own theism and theirs differ nor does he agree with them in hi
o

In [172]:
concordance_phrase(full_corpus, '', 'harmony', 'with', width=110)

 all dominions shall serve and obey him chapter in harmony with this we read in revela tions and hast made us
ng eternally this is b it should be and is more in harmony with the grandeur and in finity of the universe th
very man that cometh into the world in our text in harmony with predictions concerning himself he affirmed i 
conditions of orthodoxy our yiew of god must be in harmony with science the revelations of astronomy compelle
ous protest our view of god we ate told most be in harmony with boience true but when it is inti mated that e
 aide it is raan'a endeavour to bring himself intq harmony with the laws and principles of his own nature whi
 the world that so the world might be brought into harmony with tbis idea of god this oould not be done in a 
they are the gist of the sentence and are in exact harmony with what spiritualism has been teaching for the l
 of spiritualism however is to bring man more into harmony with the spirit world to and natural conceptions o
found in t

In [173]:
concordance_phrase(full_corpus, 'natural', 'theology', '', width=110)

broad church no church spirit o the gospel natural theology ontological haze philosophical theism the jsternal
d included we may eonridenly challenge ail natural theology ' says kant to name a single distinctive attribute
ys that the groat advance of the fu nre in natural theology is to be found in the corridor now opened upbetwee
i am ac zenophon last evening a lecture on natural theology was given by the b'shop elect of dunedin in the ha
gio il argument was well slated iv paicy's natural theology and in tho hridgo water treatises five things forc
s a storehouse of facts for the student of natural theology it will be found replete and as offering hints to 
rable of the gadarene swine with regard to natural theology i c the questions of the existence of a deity the 
ion of the strategics of religion no their natural theology may have to be reviewed their teleology recast and
a reference to what he deemed a chapter in natural theology but one not yet much handled and which perhaps can
a

In [174]:
concordance_phrase(full_corpus, '', 'huxley', '', width=110)

of bigotry and superstition spencer darwin wal lace huxley one can count up on the' fingers the little band th
ent to child was not to be realised again professor huxley in his remarkable address on f evolution and ethics
uld avail but not virtue yet virtue was evolved why huxley had some idea of the right answer when he said beca
 to the evolution of ethereal man she remarked that huxley had said that there had been a change in evolution 
ted from i he fortnightly review o july in rc iy to huxley and others is no reply whatever to mo and the fads 
i referred ray readers to pages voi ix of professor huxley s esuays prevented the possibility of mis take us t
re require it i wili explain the rages of professor huxley to which i drew attention contained statements whic
evancy whatever to the facts mentioned by professor huxley hut evidently referring to an attack on the army s 
just as reasonably ask whether bradlaugh spenser or huxley were atheists their words and works have i venture 
r

In [175]:
concordance_phrase(full_corpus, '', 'atheism', '', width=110)

y a liberal churchman he ex posed the absurdity of atheism the illogical and untenable position of agnosti ci
but would it not be better to call the subject the atheism of burns instead of burns and atheism in other wor
 subject the atheism of burns instead of burns and atheism in other words the atheism of burns s day is the s
ns instead of burns and atheism in other words the atheism of burns s day is the solid truth of to day not th
r she paints in ' donovan a oharacter who turns to atheism but finds ho cannot conscientou ly remain there an
e problems especially probloma of psychology which atheism leaves untouched ahsib besant afewsimple proofs so
nd before it could be entertained by reasoning man atheism must be proved as long as god lives he must be gre
o that is something of course better than absolute atheism but no chris tian can read the book without a cons
ain loss of children though the judge said that my atheism alone justified tbeir removal loss of friends sooi
rries in i

In [176]:
concordance_phrase(full_corpus, '', 'evolution', 'with', width=110)

rave and careful consideration of the sub ject of evolution with the view of ascertaining whether there was a
idable nature as to the supposed inconsistency of evolution with the christian belief aa to the person of chr
grave and careful consideration of tho subject of evolution with the view of ascertaining whether there was a
the compatibility or otherwiso of the doctrine of evolution with religious belief professor salmond aa it see
 modern uses of electricity were but child s play evolution with a jump dr daniel g brinton who is des cribed
speaker went on to contrast this law of spiritual evolution with the orthodox ex planation of special creatio
h exercised on the point of the reconciliation of evolution with christianity rut we believe that there never
is subject denied that he had declined to discuss evolution with him and defended at somo length tho rev jose
f the few the divine squaring of the human circle evolution with a jump dr daniel g brinton who is des cribed
 quadruped

In [177]:
concordance_phrase(full_corpus, 'of', 'evolution', '', width=110)

so he becomes convinced that the immutable law of evolution holds just as surely in the realms of light and h
t help him in hia work of develop ing a theory of evolution i must pass over the facts of geographical distri
s that would finally convince him of the truth of evolution if he still had any doubts i mean the facts of de
matter alike foe each there is the one formula of evolution from the general to the special from the simple t
tion of the facts which is known as the theory of evolution we have seen that all forms of life are connected
ch it will be seen that not only is the theory of evolution able to give the only rational account of these r
er things show that the first explanation that of evolution must be right for they occur just as evolu tion s
s they give a doable confirmation of the truth of evolution a changed world by w g p no v the bitter controve
 band that battled so heroically for that idea of evolution that wos to rise above acorn and mockery and hatr
tened to g

## Collocation

Some helper functions are necessary.

In [191]:
def filter_for_keyword(keyword):
    return lambda *w: keyword not in w
stopword_filter = lambda w: w in STOPWORDS

In [202]:
import copy
def print_collocates(keyword, bcf, measure):
    local_bcf = copy.deepcopy(bcf)
    local_bcf.apply_ngram_filter(filter_for_keyword(keyword))
    local_bcf.apply_word_filter(stopword_filter)
    local_bcf.apply_freq_filter(5)
    pmi_ws5 = local_bcf.nbest(measure.pmi, 50)
    for collocates in pmi_ws5:
        print(collocates)

In [195]:
bm = BigramAssocMeasures()

In [193]:
bcf_big = BigramCollocationFinder.from_words(full_corpus, window_size=5)

In [203]:
print_collocates('evolution', bcf_big, bm)

('evolution', 'unconditioned')
('evolution', 'unproved')
('evolution', 'compatible')
('evolution', 'unfolding')
('darwinian', 'evolution')
('theory', 'evolution')
('evolution', 'dissolution')
('factors', 'evolution')
('evolution', 'atheistic')
('evolution', 'ists')
('product', 'evolution')
('organic', 'evolution')
('hypothesis', 'evolution')
('cess', 'evolution')
('trine', 'evolution')
('gradual', 'evolution')
('cosmic', 'evolution')
('factor', 'evolution')
('darwinism', 'evolution')
('evolution', 'darwinism')
('doctrine', 'evolution')
('evolution', 'propounded')
('process', 'evolution')
('evolution', 'selection')
('chain', 'evolution')
('advocates', 'evolution')
('spontaneous', 'evolution')
('phase', 'evolution')
('accepting', 'evolution')
('doc', 'evolution')
('evolution', 'hypothesis')
('scheme', 'evolution')
('biology', 'evolution')
('believer', 'evolution')
('evolution', 'fluid')
('favour', 'evolution')
('stages', 'evolution')
('slow', 'evolution')
('evolution', 'requires')
('evol

In [204]:
terms = [
    'warfare', 
    'conflict', 
    'atheism', 
    'theism', 
    'harmony', 
    'scripture', 
    'creator', 
    'creation', 
    'huxley', # 'Darwin's Bulldog
    'kingsley', # Christian evolutionist
    'gray', # Christian evolutionist
    'agassiz', # Anti evolutionary biologist
    'paley', # Earlier natural theology
    'wilberforce', # Bishop famously debated with Darwin
    'theology', 
    'science', 
    'white', # 'Warfare' writer
    'draper' # 'Warfare' writer
]
for term in terms:
    print(f'*****TERM: {term}*****')
    print_collocates(term, bcf_big, bm)

*****TERM: warfare*****
('polemical', 'warfare')
('weapons', 'warfare')
('warfare', 'science')
*****TERM: conflict*****
('conflict', 'tween')
('conflict', 'science')
('conflict', 'opinions')
('coming', 'conflict')
('conflict', 'opinion')
('conflict', 'ideas')
('conflict', 'religion')
('race', 'conflict')
('conflict', 'religious')
('science', 'conflict')
('never', 'conflict')
('great', 'conflict')
('religion', 'conflict')
('tho', 'conflict')
('conflict', 'tho')
*****TERM: atheism*****
('sickening', 'atheism')
('deism', 'atheism')
('atheism', 'pantheism')
('mixture', 'atheism')
('atheism', 'immorality')
('theism', 'atheism')
('agnosticism', 'atheism')
('materialistic', 'atheism')
('atheism', 'materialism')
('atheism', 'agnosticism')
('professing', 'atheism')
('atheism', 'infidelity')
('fault', 'atheism')
('materialism', 'atheism')
('scepticism', 'atheism')
('tendency', 'atheism')
('practical', 'atheism')
('creed', 'atheism')
('atheism', 'gospel')
('atheism', 'christianity')
('atheism', '

('science', 'primers')
('science', 'convulsed')
('science', 'falsely')
('science', 'progressed')
('advancement', 'science')
('quarterly', 'science')
('omni', 'science')
('science', 'monthly')
('allies', 'science')
('science', 'provides')
('sical', 'science')
('victories', 'science')
('votaries', 'science')
('branches', 'science')
('modem', 'science')
('branch', 'science')
('conflict', 'science')
('frail', 'science')
('geologic', 'science')
('science', 'twin')
('science', 'pathway')
('science', 'biology')
('journal', 'science')
('physi', 'science')
('domain', 'science')
('physical', 'science')
('phy', 'science')
('experimental', 'science')
('medical', 'science')
('pursuit', 'science')
('biological', 'science')
('discoveries', 'science')
('science', 'sophy')
('reconcile', 'science')
('reconciliation', 'science')
('modern', 'science')
('antagonism', 'science')
('academy', 'science')
('contribution', 'science')
('science', 'antagonistic')
('axiom', 'science')
('science', 'astronomy')
('war

## Cooccurrence networks

Going to do cooccurrence networks again, but this time we'll attempt to have as maximal a dictionary as possible given computational resources.

In [4]:
minimum_in_docs = 5 # 10
max_prop = 0.6
dictionary = corpora.Dictionary(rel_df['Tokenised'])
dictionary.filter_extremes(no_below=minimum_in_docs, no_above=max_prop)
dictionary.compactify()

In [14]:
dictionary.save('dictionaries/rel_v2_big.dict')

In [5]:
len(dictionary)

34672

In [6]:
rel_corpus = NL_topicmodels.NL_corpus(rel_df, dictionary)
tfidf_model = TfidfModel(rel_corpus, dictionary=rel_corpus.dictionary)
rel_corpus.items['TF-IDF'] = tfidf_model[rel_corpus]

In [213]:
sparse = corpus2csc(rel_corpus.items['TF-IDF'])
dtm = pd.DataFrame.sparse.from_spmatrix(sparse)
dtm.index = rel_corpus.dictionary.values()
del sparse

In [6]:
# # Generate BOW dtm
# rel_corpus = NL_topicmodels.NL_corpus(rel_df, dictionary)
# sparse = corpus2csc(rel_corpus.items['BOW'])
# dtm = pd.DataFrame.sparse.from_spmatrix(sparse)
# dtm.index = rel_corpus.dictionary.values()
# del sparse

In [7]:
tt_df = dtm.dot(dtm.transpose())

In [8]:
dtm.to_pickle('pickles/part2_rel_dtm.tar.gz')

In [2]:
# dtm = pd.read_pickle('pickles/part2_rel_dtm.tar.gz')

In [9]:
tt_df.to_pickle('pickles/part2_rel_ttm.tar.gz')

Setting some default values and including a helper to control the style of the network.

In [13]:
philo_network = NL_helpers.network_dash(
    term='philosophy', 
    stat='mi', 
    dtm=dtm,
    ttm=tt_df,
    num_coocs=10,
    sec_coocs=4
)

In [20]:
def change_cytoscape_width(stat_choice):
    if stat_choice == 'mi':
        min = 3
        max = 7
    elif stat_choice == 'log dice':
        min = -5
        max = -1
    STYLESHEET = NL_helpers.CYTOSCAPE_STYLESHEET
    STYLESHEET[0]['style']['width'] = f'mapData(weight, {min}, {max}, 1, 5)'
    return STYLESHEET

In [22]:
app = JupyterDash(__name__)

philo_cytoscape = cyto.Cytoscape(
        id='philosophy-network',
        minZoom=1,
        layout={'name': 'cose'},
        style={'width': '85%', 'height': '800px',
            'margin': 'auto', 'border-style': 'solid',
            'margin-top': '10px'},
        elements=philo_network,
        stylesheet=[
            {
                'selector': 'edge',
                'style': {
                    'width': 'mapData(weight, 3, 6, 1, 3)',
                    'line-color': 'silver'
                }
            },
            {
                'selector': 'node',
                'style': {
                    'content': 'data(label)',
                    'width': 'mapData(size, 1, 10, 10, 20)',
                    'height': 'mapData(size, 1, 10, 10, 20)'
                }
            },
            {
                'selector': 'label',
                'style': {
                    'font-size': 6,
                    'text-valign': 'center',
                    'text-background-color': 'white',
                    'text-background-opacity': 0.6,
                    'text-background-padding': 1,
                    'text-border-color': 'black',
                    'text-border-opacity': 1,
                    'text-border-width': 0.5
                }
            }
        ]
    )

app.layout = html.Div([
    html.H2("Cooccurence Networks (TF-IDF)"),
    html.P("Search Term:"),
    dcc.Input(
        id='search-term',
        type='text',
        value='philosophy'
    ),
    html.P("Statistic:"),
    dcc.Dropdown(
        id='stat-choice',
        options=[
            {'label': 'Mutual likelihood', 'value': 'mi'},
            {'label': 'Log Dice', 'value': 'log dice'}
        ],
        value='mi'
    ),
        html.P('Primary Cooccurences'),
    dcc.Slider(
        id='primary-coocs',
        min=1,
        max=50,
        step=1,
        value=15,
        marks = {n: f'{n}' for n in [i for i in range(1, 51) if i%5==0]}
    ),
    html.P('Secondary Cooccurences'),
    dcc.Slider(
        id='secondary-coocs',
        min=1,
        max=15,
        step=1,
        value=5,
        marks = {n: f'{n}' for n in [i for i in range(1, 51) if i%5==0]}
    ),
    html.Button('Submit', id='submit-val', n_clicks=0),
    philo_cytoscape
])

@app.callback(
    [Output(component_id='philosophy-network', component_property='elements'),
    Output(component_id='philosophy-network', component_property='stylesheet')],
    [Input(component_id='submit-val', component_property='n_clicks')],
    [State(component_id='search-term', component_property='value'),
    State(component_id='stat-choice', component_property='value'),
    State(component_id='primary-coocs', component_property='value'),
    State(component_id='secondary-coocs', component_property='value')]
)
def update_network(n_clicks, search_term, stat_value, primary_coocs, secondary_coocs):
    network = NL_helpers.network_dash(
        term=search_term, 
        stat=stat_value, 
        dtm=dtm,
        ttm=tt_df,
        num_coocs=primary_coocs,
        sec_coocs=secondary_coocs
    )
    stylesheet = change_cytoscape_width(stat_value)
    return network, stylesheet


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


## Odds and ends

**An objection:** this is all *history* rather than *philosophy*.

**Reponse:** At the point when evolution comes on the scene, the scientists and clergy have to become philosophers. '...with the publication of the *Origin* there is a period of intese excitement, and scientists who have never risen above the level of taxonomic detail begin to talk of philosophy, lreigion, the nature of man, and the shape of the cosmos. Some of this sudden theorizing is brilliant and insightful, some is clumsy and inept, but almost never is discourse informed by the rigor of the professional philosopher.' (Dupree, 353).

A possible extension using the corpus: see how this understanding changed over time.